# Control Calidad del Array
Autores: Rafaella Ormond y Jose Jaime Martinez-Magana <br>
***Descripción:***<br>
Este script preparará los archivos para realizar la imputación

**INPUT:** Archivos PLINK (.bed/.bim/.fam)<br>
**OUTPUT:** Archivos VCF para enviar al servidor de imputación TOPMed

### ***Requisitos:***
### Descargar Plink
Podemos descargar Plink versión 1.9 y versión 2.0 siguiendo los pasos que se indican en su sitio web.<br>
Para instalar plink2 [access here](https://www.cog-genomics.org/plink/2.0/)<br>
Para instalar plink1.9 [access here](https://www.cog-genomics.org/plink/1.9/) <br>

### Descargar BCF tools:
Para instalar bcftools [access here](http://samtools.github.io/bcftools/) <br>

### Genoma de Referencia: 
Para los primeros pasos, el genoma de referencia depende de la matriz, se puede utilizar hg19 o hg38<br>
Para descargar el genoma de referencia hg19, descargue el archivo: https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz<br> 
Para descargar el genoma de referencia hg38, descargue el archivo: https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

### Imputación en TOPMed
Después de ejecutar este script, la imputación debe realizarse en TOPMed [link here](https://imputation.biodatacatalyst.nhlbi.nih.gov/#!)<br>

**Nota:** Después de la imputación, todos los archivos deben alinearse con el genoma de referencia **hg38** para garantizar la coherencia.

### Pasos del Análisis:
1. **Configuración de Usuario** – Defina rutas, genoma de referencia y parámetros del software.
2. **Convertir PLINK a VCF** – Convertir `.bed/.bim/.fam` archivos a `.vcf` formatearlos e indexarlos.
3. **Alineación** – Alinear genotipos con el genoma de referencia (verificación de cadenas).
4. **Dividir por Cromosoma** – Genera un archivo VCF por cada cromosoma (`chr1.vcf.gz`, `chr2.vcf.gz`, ...).
5. **Instrucciones para la imputación en TOPMED** - Genera un archivo imputado en hg38 y phased.

### 1) Configuración del Usuario
***Descripción:***<br>
Establece los parámetros y ajusta la ruta y los archivos según tu análisis

In [ ]:
# CONFIGURACIÓN DEL USUARIO - MODIFICA SEGÚN TU ANÁLISIS

# Prefijo de archivos PLINK (.bed, .bim, .fam)
inPath="/ruta_a_tus_datos/prefijo_plink"

# Nombre de tu cohorte
cohortName="nombre_cohorte"

# Directorio para archivos VCF
clvcfPath="ruta_archivos_vcf"

# Directorio para VCFs por cromosoma
clvcfbychrPath="ruta_vcf_cromosoma"

# Archivo de genoma de referencia (hg19 o hg38)
refFile="/ruta/al/referencia/hg19.fa"

### 2) Convertir archivos PLINK a VCF, realizar control de calidad e indexar los archivos
***Descripción:***<br>
Convierte archivos binarios PLINK a formato VCF comprimido y crea archivos de índice para acceso eficiente.
Se aplican filtros de control de calidad durante la conversión para excluir variantes y muestras según la frecuencia alélica menor (MAF), equilibrio de Hardy-Weinberg (HWE), tasa de genotipos faltantes por variante y por individuo.

Filtros de control de calidad: <br>
--maf 0.01  → excluye variantes con frecuencia alélica menor < 1%<br>
--hwe 1e-6  → excluye variantes con p-valor de Hardy-Weinberg < 1e-6<br>
--geno 0.05 → excluye variantes con tasa de genotipos faltantes > 5%<br>
--mind 0.05 → excluye muestras con tasa de genotipos faltantes > 5%<br>

Por favor, ajusta las rutas y nombres de archivos según corresponda.

In [ ]:
# Convierte archivos binarios PLINK (.bed/.bim/.fam) a un VCF comprimido (.vcf.gz) y aplica filtros de control de calidad
# - 'bgz' genera un archivo VCF comprimido por bloques (bgzip) compatible con bcftools/tabix
# La opción 'vcf-iid bgz' elimina el campo Family ID (FID), manteniendo solo el Individual ID (IID)
plink2 --bfile ${inPath} --chr 1-22 --recode vcf-iid bgz --maf 0.01 --hwe 1e-6 --geno 0.05 --mind 0.05 --out ${clvcfPath}/${cohort_name}

# Indexa el archivo VCF resultante usando bcftools
# Esto crea un archivo de índice .csi, necesario para acceso rápido y análisis posteriores
bcftools index ${clvcfPath}/${cohort_name}.vcf.gz

### 3) Alineación
***Descripción:***<br>
Alinea los alelos del VCF con el genoma de referencia<br>
Ajusta el número de hilos según corresponda.

In [ ]:
# Convierte el archivo VCF a formato BCF usando 25 hilos para procesamiento más rápido (ajusta según tu capacidad computacional)
bcftools convert ${clvcfPath}/${cohort_name}.vcf.gz -Ob -o ${clvcfPath}/${cohort_name}.bcf.gz --threads 25

# Ejecuta el plugin fixref de BCFtools para corregir inconsistencias entre alelos de referencia/alternativos
# Este paso verifica y actualiza los alelos según el genoma de referencia
bcftools +fixref ${clvcfPath}/${cohort_name}.bcf.gz -- -f ${refFile}

# Ejecuta fixref nuevamente, esta vez generando un nuevo archivo VCF
# La opción -d elimina alelos que no coinciden con la referencia
# La opción -m flip invierte alelos para que coincidan con la referencia cuando sea posible
# El resultado final es un archivo VCF corregido para desajustes de referencia
bcftools +fixref ${clvcfPath}/${cohort_name}.bcf.gz -Oz -o ${clvcfPath}/${cohort_name}.vcf.gz -- -d -m flip -f ${refFile}

### 4) Dividir archivo VCF por cromosoma
***Descripción:***<br>
Divide el archivo VCF por cromosoma para la imputación y ordena los archivos VCF por cromosoma para asegurar el orden correcto antes de los análisis posteriores

In [ ]:
# Itera sobre los números de cromosoma
for chr_num in {1..22}
do
    plink2 --vcf ${clvcfPath}/${cohort_name}.vcf.gz --chr ${chr_num} --recode vcf-4.2 bgz --out ${clvcfbychrPath}/${cohort_name}_forimputation_chr${chr_num}
done

# Ordenamiento
# Itera sobre los números de cromosoma
for chr_num in {1..22}
do
    bcftools sort ${clvcfbychrPath}/${cohort_name}_forimputation_chr${chr_num}.vcf.gz -Oz -o ${clvcfbychrPath}/${cohort_name}_forimputation_chr${chr_num}_sorted.vcf.gz
done

## 5) Instrucciones para la Imputación en TOPMed

Accede al [Servidor de Imputación TOPMed](https://imputation.biodatacatalyst.nhlbi.nih.gov).

Inicia sesión en tu cuenta y accede al panel principal.
En la parte superior de la página, haz clic en **“Run” → “Genotype Imputation (Minimac4)”**.

![Ejemplo de interfaz TOPMed](./topmed1.png)

Sube tus archivos VCF y configura las siguientes opciones:

- **Array Build:** selecciona *hg19* o *hg38* (según la construcción de tu archivo).
- **Rsq Filter:** *off*  
- **Phasing:** *Eagle v2.4 (phased output)*  
- **Population:** selecciona *Todas las poblaciones* (actualmente “vs. TOPMed Panel”)  
- **Mode:** *Control de Calidad & Imputación*  
- Activa la casilla **“Generate Meta-imputation file”**

![Ejemplo de interfaz TOPMed](./topmed2.png)

Después de enviar el trabajo, recibirás un correo electrónico cuando los resultados estén listos.
También recibirás una **contraseña por correo** para acceder a las carpetas de salida.
Descarga todas las carpetas de resultados a tu directorio de trabajo.
